<a href="https://colab.research.google.com/github/onlymid/yfinance/blob/main/course_250.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![](https://mermaid.ink/img/pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8?type=png)](https://mermaid.live/edit#pako:eNptkc1Kw0AYRV9lmHX6AlkIbdPWjStdOcliaKZNoPkhThaSFEqlJdhNQF2IEolQUYTiX6WCxZdxkvEtTJNaIzirOd89DMN3Pdi2VAJF2HWwrYE9STZBdqroc_mRnt7ywYi_L9hoqoBKZQvUULpYprM4Dceg2iUmVQq9lqd1jwXjJAr5_SR5HH7FEYsm_UKorwSfD2N29JZcTfndtQ8kVGb2ELDZq1LWk-MBj0-KN33QQGX-R-dPMZuc_-hNVOY_popT_toWSsxcWBik8zCJLtdRo4gKaJaRlcM2YhcRGz6n85vNghQoQIM4BtbVbI_eSpch1YhBZChmV5V0sNujMpTJfqZil1q7h2YbitRxiQBdW8WUSDpOGjCg2MG9g820oerUcjZDG5v7lvUrkTzeKQrMexSgY7ldbW30v0kGPk8)

In [1]:
%%capture
!pip install google-generativeai
!pip install yfinance pandas_ta

In [3]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import os
import google.generativeai as genai
from IPython.display import Markdown

In [4]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [5]:
# 股价走势查询函数
def get_stock_price_history(symbol, period="1mo"):
    try:
        stock = yf.Ticker(symbol)
        history = stock.history(period=period)
        return history.tail().to_dict('records')
    except Exception as e:
        return f"无法获取 {symbol} 的历史价格: {str(e)}"

In [6]:
# 股票信息查询函数
def get_stock_info(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        return {
            '公司名称': info.get('longName', 'N/A'),
            '现价': info.get('currentPrice', 'N/A'),
            '市值': info.get('marketCap', 'N/A'),
            '产业': info.get('industry', 'N/A'),
            '52周高点': info.get('fiftyTwoWeekHigh', 'N/A'),
            '52周低点': info.get('fiftyTwoWeekLow', 'N/A')
        }
    except Exception as e:
        return f"无法获取 {symbol} 的信息: {str(e)}"

In [7]:
# 财务数据查询函数
def get_financial_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        financials = stock.financials.iloc[:, 0]
        balance_sheet = stock.balance_sheet.iloc[:, 0]
        return {
            '营收': financials.get('Total Revenue', 'N/A'),
            '净利': financials.get('Net Income', 'N/A'),
            '总资产': balance_sheet.get('Total Assets', 'N/A'),
            '总负债': balance_sheet.get('Total Liabilities Net Minority Interest', 'N/A'),
            'ROE': stock.info.get('returnOnEquity', 'N/A'),
            '本益比': stock.info.get('trailingPE', 'N/A')
        }
    except Exception as e:
        return f"无法获取 {symbol} 的财务数据: {str(e)}"

In [8]:
# 技术分析函数
def get_technical_analysis(symbol, period="6mo"):
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period=period)

        # 计算技术指标
        df['RSI'] = ta.rsi(df['Close'], length=14)
        macd = ta.macd(df['Close'])
        df = pd.concat([df, macd], axis=1)
        df['SMA_20'] = ta.sma(df['Close'], length=20)
        df['SMA_60'] = ta.sma(df['Close'], length=60)
        stoch = ta.stoch(df['High'], df['Low'], df['Close'])
        df = pd.concat([df, stoch], axis=1)

        latest = df.iloc[-1]
        return {
            'indicators': {
                'RSI': round(latest['RSI'], 2),
                'MACD': round(latest['MACD_12_26_9'], 2),
                'MACD_Signal': round(latest['MACDs_12_26_9'], 2),
                'SMA_20': round(latest['SMA_20'], 2),
                'SMA_60': round(latest['SMA_60'], 2),
                'K值': round(latest['STOCHk_14_3_3'], 2),
                'D值': round(latest['STOCHd_14_3_3'], 2),
                '收盘价': round(latest['Close'], 2)
            },
            'historical_data': df.tail().to_dict('records')
        }
    except Exception as e:
        return f"无法进行技术分析: {str(e)}"

In [11]:
# Gemini 专业分析函数
def analyze_with_gemini(query, context):
    model = genai.GenerativeModel('gemini-2.0-pro-exp-02-05')
    prompt = f"""
    你是一个专业的金融分析助手，请根据以下数据回答用户问题：
    上下文数据：{context}
    用户问题：{query}
    请用专业且易懂的方式进行分析，并给出投资建议。
    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"API请求失败: {str(e)}"

In [ ]:
# 主交互系统
def main():
    print("欢迎使用股票市场助手（Gemini版）！")
    print("可查询功能：实时报价/财务数据/技术分析/综合评估")

    while True:
        symbol = input("\n请输入股票代码 (e.g. 2330.TW) 或输入exit退出: ")
        if symbol.lower() in ['exit', 'quit']:
            break

        # 获取基础数据
        price_data = get_stock_price_history(symbol)
        info_data = get_stock_info(symbol)
        financial_data = get_financial_data(symbol)
        technical_data = get_technical_analysis(symbol)

        # 构建分析上下文
        context = {
            '基本信息': info_data,
            '财务数据': financial_data,
            '技术指标': technical_data.get('indicators', {}),
            '历史数据': technical_data.get('historical_data', [])
        }

        # 获取用户问题
        query = input("请输入您的分析问题 (e.g. 技术面分析/财务健康度/综合评估): ")

        # 使用Gemini进行分析
        analysis = analyze_with_gemini(query, context)
        display(Markdown(f"**Gemini分析结果:**\n{analysis}"))

if __name__ == "__main__":
    main()

欢迎使用股票市场助手（Gemini版）！
可查询功能：实时报价/财务数据/技术分析/综合评估

请输入股票代码 (e.g. 2330.TW) 或输入exit退出: 600888.SS
请输入您的分析问题 (e.g. 技术面分析/财务健康度/综合评估): 综合


**Gemini分析结果:**
好的，下面为您综合分析Xinjiang Joinworld Co., Ltd. (新疆众和) 的投资前景：

**公司概况与财务健康状况**

*   **行业地位：** 新疆众和属于铝产业，这是一个周期性较强的行业，容易受到宏观经济和市场供需关系的影响。
*   **规模与市值：** 公司市值约100亿人民币，在行业内属于中等规模。
*   **盈利能力：**
    *   营收超过65亿，净利润超过15亿，显示出公司良好的盈利能力。
    *   净资产收益率（ROE）为12.7%，表明公司利用股东资金创造利润的效率较好。
    *   市盈率（P/E）约为7.79，相对于其盈利能力，估值相对较低。
*   **偿债能力：** 总资产超过167亿，总负债约70亿，资产负债率约为42%，财务结构相对稳健。

**技术指标分析**

*   **短期趋势：**
    *   相对强弱指数（RSI）为64.43，接近超买区域（通常为70以上），可能存在短期回调风险。
    *   平滑异同移动平均线（MACD）为正值，且MACD线高于信号线，表明短期动能积极。
    *   20日均线和60日均线均为正值，且股价高于均线，表明股价处于上升趋势。
*   **随机指标：** K值和D值均高于70，处于较高水平，也提示短期内可能存在回调。

**历史数据观察**

*   最近几个交易日，股价在小幅波动中上涨，成交量有所放大，显示市场对该股的关注度较高。

**投资建议**

综合来看，新疆众和基本面良好，财务状况稳健，盈利能力较强，且估值相对合理。技术指标显示短期内股价处于上升趋势，但存在一定的回调风险。

**投资策略：**

1.  **中长期投资者：** 如果您看好铝行业的前景，并且能够承受一定的周期性波动，可以考虑将新疆众和纳入您的投资组合。鉴于其良好的基本面和较低的估值，可以逢低逐步买入，并长期持有。
2.  **短期投资者：** 如果您更关注短期交易机会，可以密切关注技术指标的变化。目前RSI和随机指标均处于较高水平，可以等待回调至合理价位（如20日均线附近）再考虑买入。同时，设置止损位以控制风险。

**风险提示：**

*   **行业风险：** 铝行业受宏观经济和市场供需影响较大，价格波动可能较为剧烈。
*   **政策风险：** 环保政策、产业政策等变化可能对公司经营产生影响。
*   **市场风险：** 股市整体走势、投资者情绪等因素可能导致股价波动。

**免责声明：**

本分析仅供参考，不构成任何投资建议。投资者应根据自身风险承受能力和投资目标，独立做出决策。
